In [3]:
import pandas as pd
from connection import connect

In [24]:
#connection
SCHEMA='sales'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

['sales_person_quota_history', 'country_region_currency', 'sales_reason', 'credit_card', 'sales_tax_rate', 'currency', 'sales_territory', 'currency_rate', 'sales_territory_history', 'customer', 'shopping_cart_item', 'person_credit_card', 'special_offer', 'special_offer_product', 'sales_order_header_sales_reason', 'sales_order_detail', 'sales_order_header', 'sales_person', 'store']


## Extract

In [25]:
dim_sales_territory=pd.read_sql_table('sales_territory',co_oltp,SCHEMA)

dim_sales_territory.head(5)

,territory_id,name,country_region_code,group_name,sales_ytd,sales_last_year,cost_ytd,cost_last_year,rowguid,modified_date
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30 00:00:00+00:00
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30 00:00:00+00:00
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30 00:00:00+00:00
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30 00:00:00+00:00
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30 00:00:00+00:00


## Transform


In [26]:

dim_sales_territory['sales_territory_country']=dim_sales_territory['name']
dim_sales_territory['sales_territory_country']=dim_sales_territory.apply(lambda row:'United States' if row['country_region_code']=='US' else row['sales_territory_country'],axis=1)
dim_sales_territory.drop(['country_region_code','sales_ytd','sales_last_year','cost_ytd','cost_last_year','rowguid','modified_date'],axis=1,inplace=True)

dim_sales_territory.rename(columns={'name':'sales_territory_region',
                                    'territory_id':'sales_territory_alternate_key',
                                     'group_name':'sales_territory_group'                                   
                                    },inplace=True)


dim_sales_territory

,sales_territory_alternate_key,sales_territory_region,sales_territory_group,sales_territory_country
0,1,Northwest,North America,United States
1,2,Northeast,North America,United States
2,3,Central,North America,United States
3,4,Southwest,North America,United States
4,5,Southeast,North America,United States
5,6,Canada,North America,Canada
6,7,France,Europe,France
7,8,Germany,Europe,Germany
8,9,Australia,Pacific,Australia
9,10,United Kingdom,Europe,United Kingdom


In [27]:

# dim_sales_territory_or=pd.read_sql_table('dim_sales_territory',etl_conn_or)

# for index,row in dim_sales_territory_or.iterrows():
#     image_data=row['sales_territory_image']

#     if image_data is not None:
#         with open(f'../images/usa_flag_{row['sales_territory_alternate_key']}.png', 'wb') as file:
#             file.write(image_data)

def get_sales_territory_image(row):

    filename_path=f"../images/usa_flag_{row['sales_territory_alternate_key']}.png"
    with open(filename_path,'rb') as f:
        image_bytes=f.read()
    return image_bytes

dim_sales_territory['sales_territory_image']=dim_sales_territory.apply(get_sales_territory_image,axis=1)




dim_sales_territory
    

,sales_territory_alternate_key,sales_territory_region,sales_territory_group,sales_territory_country,sales_territory_image
0,1,Northwest,North America,United States,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
1,2,Northeast,North America,United States,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
2,3,Central,North America,United States,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
3,4,Southwest,North America,United States,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
4,5,Southeast,North America,United States,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
5,6,Canada,North America,Canada,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
6,7,France,Europe,France,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
7,8,Germany,Europe,Germany,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
8,9,Australia,Pacific,Australia,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...
9,10,United Kingdom,Europe,United Kingdom,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...


## Load

In [28]:
dim_sales_territory.to_sql('dim_sales_territory',etl_conn,if_exists='append',index=False)

10